In [52]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import unicodedata
import os
import webbrowser
import html5lib
from openpyxl import workbook
from datetime import datetime
import requests
from fuzzywuzzy import process
import time
from icecream import ic

In [53]:
path = '../Master_tables_GSBD_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, keep_default_na=False,
                           na_values=['_'], sheet_name=None)


dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: September 30 2023 : (425, 18)
DataFrame name: June 30 2023 : (413, 16)
DataFrame name: March 31 2023 : (395, 16)
DataFrame name: December 31 2022 : (345, 16)
DataFrame name: September 30 2022 : (341, 16)
DataFrame name: June 30 2022 : (382, 33)
DataFrame name: March 31 2022 : (321, 16)
DataFrame name: December 31 2021 : (365, 16)
DataFrame name: September 30 2021 : (318, 16)
DataFrame name: June 30 2021 : (312, 18)
DataFrame name: March 31 2021 : (307, 19)
DataFrame name: December 31 2020 : (49, 17)
DataFrame name: September 30 2020 : (246, 16)
DataFrame name: June 30 2020 : (229, 11)
DataFrame name: March 31 2020 : (236, 8)
DataFrame name: December 31 2019 : (200, 25)
DataFrame name: September 30 2019 : (225, 23)
DataFrame name: June 30 2019 : (213, 23)
DataFrame name: March 31 2019 : (176, 21)
DataFrame name: December 31 2018 : (166, 21)
DataFrame name: September 30 2018 : (153, 23)
DataFrame name: June 30 2018 : (129, 23)
DataFrame name: March 31 2018 : (81, 25)
Data

In [54]:
# def timer(func):
#     def wrapper():
#         t1 = time.time()
#         func()
#         t2 = time.time() - t1
#         print(f'{func.__name__} takes {t2} sec to run')
#     return wrapper

In [55]:
process_tables = {}
process_tables_shape = {}

if not os.path.exists("../PT_csv_files"):
    os.mkdir('../PT_csv_files')

headers = {}


def run_process_function(dataframes, process_tables, process_tables_shape):
    path = '../process_tables_GSBD_Investment.xlsx'
    writer = pd.ExcelWriter(path=path, engine='openpyxl')
    for dataframe in dataframes:
        print(dataframe)
        processed_table = process_table_function(dataframes[dataframe])
        process_tables[dataframe] = processed_table
        process_tables_shape[dataframe] = processed_table.shape
        headers[dataframe] = processed_table.columns.values
        processed_table.to_excel(
            writer, sheet_name=dataframe.replace(',', ''), index=False)
        processed_table.to_csv(
            '../PT_csv_files/'+dataframe.replace(',', '')+'.csv')

        writer.book.save(path)
    writer.close()


def shape(count, df) -> int:
    print(f"{count} : shape : {df.shape}")
    count += 1
    return count


def dropna_col_row(df) -> pd.DataFrame:
    df = df.dropna(how='all', axis=0).reset_index(drop=True)
    df = df.dropna(how='all', axis=1).reset_index(drop=True)
    return df


def drop_if_contain(pattern, df) -> pd.DataFrame:
    matching_rows = df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    df = df[~matching_rows]
    return df

In [56]:
dataframes.pop('December_31_2020')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ITEM 1.,FINANCIAL STATEMENTS,,,,,,,,,,,,,,,
1,ITEM 1.,FINANCIAL STATEMENTS,,,,,,,,,,,,,,,
2,Investment *,Industry,InterestRate (+),Reference Rate andSpread (+),Floor (+),,Maturity,ParAmount/Shares(++),,Cost,,Fair Value,,,,,
3,1st Lien/Senior Secured Debt* #- 156.46,,,,,,,,,,,,,,,,
4,1272775 B.C. LTD. (dba Everest Clinical Resear...,Professional Services,6.50,L + 5.50,1.00,,11/06/26,,9430,,,9292,,,9289,,
5,1272775 B.C. LTD. (dba Everest Clinical Resear...,Professional Services,,L + 5.50,1.00,,11/06/26,,1260,,,(18,,,(19,,
6,"3SI Security Systems, Inc.(4)",Commercial Services & Supplies,6.75,L + 5.75,1.00,,06/16/23,,14556,,,14465,,,14483,,
7,"3SI Security Systems, Inc.(4)",Commercial Services & Supplies,6.75,L + 5.75,1.00,,06/16/23,,2216,,,2075,,,2205,,
8,"A Place For Mom, Inc.",Diversified Consumer Services,4.75,L + 3.75,1.00,,08/10/24,,8752,,,8746,,,8052,,
9,"Acquia, Inc.(2) (4)",Software,8.00,L + 7.00,1.00,,10/31/25,,30561,,,28977,,,30026,,


In [75]:

def process_table_function(soi_table_df) -> pd.DataFrame:
    count = 1
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.replace(
        r'^\s*\$\s*$', '', regex=True).replace(r'\n', '', regex=True)
    soi_table_df = soi_table_df.replace('-', '0')
    soi_table_df = soi_table_df.replace('€', np.nan)
    soi_table_df = soi_table_df.replace('C$', np.nan)
    soi_table_df = soi_table_df.replace('£', np.nan)
    soi_table_df = soi_table_df.replace('CAD', np.nan)
    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.apply(
        lambda x: x.strip() if isinstance(x, str) else x)
    count = shape(count, soi_table_df)

    # drops all the extra top row
    pattern = r'Industry'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax():].reset_index(
            drop=True)
    count = shape(count, soi_table_df)

    # drops all the extra bottom row
    pattern = r'Total\s+Investments | Total\s+Common\s+Stock'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row

    # if soi_table_df[matching_rows].index[0] < 20:
    #     soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
    #         drop=True)
    # else:
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[:matching_rows.idxmax()].reset_index(
            drop=True)

    # soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
    #     drop=True)
    count = shape(count, soi_table_df)

    count = shape(count, soi_table_df)

    return soi_table_df


run_process_function(dataframes=dataframes, process_tables=process_tables,
                     process_tables_shape=process_tables_shape)

September_30_2023
1 : shape : (425, 18)
2 : shape : (425, 18)
3 : shape : (425, 18)
4 : shape : (425, 18)
5 : shape : (419, 18)
6 : shape : (419, 18)
June_30_2023
1 : shape : (413, 16)
2 : shape : (413, 16)
3 : shape : (413, 16)
4 : shape : (413, 16)
5 : shape : (407, 16)
6 : shape : (407, 16)
March_31_2023
1 : shape : (395, 16)
2 : shape : (395, 16)
3 : shape : (395, 16)
4 : shape : (395, 16)
5 : shape : (389, 16)
6 : shape : (389, 16)
December_31_2022
1 : shape : (345, 16)
2 : shape : (345, 16)
3 : shape : (345, 16)
4 : shape : (345, 16)
5 : shape : (339, 16)
6 : shape : (339, 16)
September_30_2022
1 : shape : (341, 16)
2 : shape : (341, 16)
3 : shape : (341, 16)
4 : shape : (341, 16)
5 : shape : (337, 16)
6 : shape : (337, 16)
June_30_2022
1 : shape : (382, 33)
2 : shape : (382, 33)
3 : shape : (382, 33)
4 : shape : (382, 33)
5 : shape : (328, 33)
6 : shape : (328, 33)
March_31_2022
1 : shape : (321, 16)
2 : shape : (321, 16)
3 : shape : (321, 16)
4 : shape : (321, 16)
5 : shape : (